In [1]:
import cv2
import numpy as np
import os
import caffe
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#图像保持比例放大或缩小
class PreprocessResizeKeepRatio(object):

    def __init__(self, width, height):
        self.width = width
        self.height = height

    def do(self, cv2_img):
        max_width = self.width
        max_height = self.height

        cur_height, cur_width = cv2_img.shape[:2]#当前图像尺寸
        if float(cur_width)!=0 and float(cur_height)!=0:
            ratio_w = float(max_width)/float(cur_width)
            ratio_h = float(max_height)/float(cur_height)
            ratio = min(ratio_w, ratio_h)

            new_size = (min(int(cur_width*ratio), max_width),
                        min(int(cur_height*ratio), max_height))

            new_size = (max(new_size[0], 1),
                        max(new_size[1], 1),)

            resized_img = cv2.resize(cv2_img, new_size)
            return resized_img
        return cv2_img

In [3]:
#取峰值
def extract_peek_ranges_from_array_x(array_vals, minimun_val=20000, minimun_range=150):
    """
    start_i = None
    end_i = None
    peek_ranges = []
    for i, val in enumerate(array_vals):
        if val > minimun_val and start_i is None:
            start_i = i
        if val > minimun_val:
            end_i = i
                      
    peek_ranges.append((start_i, end_i))    
    return peek_ranges
    """
    start_i = None
    end_i = None
    peek_ranges = []
    for i, val in enumerate(array_vals):
        if val > minimun_val and start_i is None:
            start_i = i
        elif val > minimun_val and start_i is not None:
            #end_i = i
            pass
        elif val < minimun_val and start_i is not None:
            end_i = i
            if end_i - start_i <= minimun_range:
                start_i = None
                end_i = None
                #peek_ranges.append((start_i, end_i))
            else:
                end_i = i
                break;
        elif val < minimun_val and start_i is None:
            pass
        else:
            raise ValueError("cannot parse this case...")
    if end_i == None:
        end_i = 250
    peek_ranges.append((start_i, end_i))
    return peek_ranges


def extract_peek_ranges_from_array_y(array_vals, minimun_val=2000, minimun_range=10):
    left_i = None
    right_i = None
    start_i = None
    end_i = None
    length = len(array_vals)
    peek_ranges = []
    for i, val in enumerate(array_vals):
        if val > minimun_val and start_i is None:
            start_i = i
        elif val > minimun_val and start_i is not None:
            pass
        elif val < minimun_val and start_i is not None:
            end_i = i
            if end_i - start_i <= minimun_range:
                start_i = None
                end_i = None
                #peek_ranges.append((start_i, end_i))
            else:
                left_i = start_i
                break;
        elif val < minimun_val and start_i is None:
            pass
        else:
            raise ValueError("cannot parse this case...")

    for i, val in enumerate(array_vals[::-1]):
        if val > minimun_val and start_i is None:
            start_i = i
        elif val > minimun_val and start_i is not None:
            pass
        elif val < minimun_val and start_i is not None:
            end_i = i
            if end_i - start_i <= minimun_range:
                start_i = None
                end_i = None
                #peek_ranges.append((start_i, end_i))
            else:
                right_i = length - start_i
                break;
        elif val < minimun_val and start_i is None:
            pass
        else:
            raise ValueError("cannot parse this case...")

    #peek_ranges.append((left_i, right_i))
    if right_i == None:
        right_i = 980
    if left_i == None:
        left_i = 20
    width = int((right_i-left_i)/7.5)
    for i in range(2):
        start_i = left_i+width*i
        end_i = left_i+width*(i+1)
        peek_ranges.append((start_i, end_i))
        # peek_ranges.append((start_i-((i+1)*2), end_i-((i+1)*2)))
    for i in range(5):
        start_i = right_i-width*(i+1)
        end_i = right_i-width*i
        peek_ranges.append((start_i-(i*2), end_i-(i*2)))
    return peek_ranges

def extract_peek_ranges_from_array_single(array_vals, minimun_val=1000, minimun_range=10):
    start_i = None
    end_i = None
    peek_ranges = []
    for i, val in enumerate(array_vals):
        if val > minimun_val and start_i is None:
            start_i = i
        elif val > minimun_val and start_i is not None:
            #end_i = i
            pass
        elif val < minimun_val and start_i is not None:
            end_i = i
            if end_i - start_i <= minimun_range:
                start_i = None
                end_i = None
                #peek_ranges.append((start_i, end_i))
            else:
                end_i = i
                break;
        elif val < minimun_val and start_i is None:
            pass
        else:
            raise ValueError("cannot parse this case...")
    if end_i == None:
        end_i = 250
    peek_ranges.append((start_i, end_i))
    return peek_ranges



In [4]:
## Draw
def draw_rectangle(color_img, peek_ranges, vertical_peek_ranges2d):
    
    color = (255, 0, 0)
    for i, peek_range in enumerate(peek_ranges):#行
        for vertical_range in vertical_peek_ranges2d[i]:#列
            x = vertical_range[0]
            y = peek_range[0]
            w = vertical_range[1] - x
            h = peek_range[1] - y
            pt1 = (x, y)
            pt2 = (x + w, y + h)
            cv2.rectangle(color_img, pt1, pt2, color)

In [5]:
## Try to find text lines and chars
def crop_direct(peek_ranges, adaptive_image, debug = False):   
    vertical_peek_ranges2d = []
    for peek_range in peek_ranges:
        start_y = peek_range[0]
        end_y = peek_range[1]
        line_img = adaptive_image[start_y:end_y, :]
        vertical_sum = np.sum(line_img, axis=0)
        vertical_peek_ranges = extract_peek_ranges_from_array_y(
            vertical_sum)
        #vertical_peek_ranges = median_split_ranges(vertical_peek_ranges)#利用中位数再次切割     
        vertical_peek_ranges2d.append(vertical_peek_ranges)
    return vertical_peek_ranges2d

In [6]:
### 求一行的中位数宽度
def compute_median_w_from_ranges(peek_ranges):
    widthes = []
    for peek_range in peek_ranges:
        w = peek_range[1] - peek_range[0] + 1
        widthes.append(w)
    widthes = np.asarray(widthes)
    median_w = np.median(widthes)
    return median_w

In [7]:
def is_symbol(num_image, array_vals, cur_h, minimun_val=3, h_ratio=0.4):
    start_i = None
    end_i = None
    for i, val in enumerate(array_vals):
        if val > minimun_val:      
            if i > cur_h*h_ratio:
                return True
            else:
                return False

In [8]:
def remove_noise(peek_ranges, vertical_peek_ranges2d, adaptive_image, stan_w_ratio = 0.4, debug = True):#ratio:
    filtered_vertical_peek_ranges2d = []
    num_image = 0
    for i, peek_range in enumerate(peek_ranges):
        new_peek_range = []
        median_w = compute_median_w_from_ranges(vertical_peek_ranges2d[i])
        stan_w = median_w * stan_w_ratio
        cur_h = peek_range[1] - peek_range[0]
        for j, vertical_range in enumerate(vertical_peek_ranges2d[i]):
            cur_w = vertical_range[1] - vertical_range[0]
            if cur_w < stan_w and cur_w > 0.1*median_w:
                char_image = adaptive_image[peek_range[0]:peek_range[1], vertical_range[0]:vertical_range[1]]          
                horizontal_sum = np.sum(char_image, axis=1)
                num_image += 1
                if debug:
                    plt.subplot(20, 10, num_image)
                    plt.imshow(char_image)
                if is_symbol(num_image, horizontal_sum, cur_h):
                    plt.title(str(num_image)+':True')
                else:
                    plt.title(str(num_image)+':False')
                    new_peek_range.append(vertical_range)                              
            elif cur_w >= stan_w:
                new_peek_range.append(vertical_range)
            else:
                pass
        filtered_vertical_peek_ranges2d.append(new_peek_range)
    return filtered_vertical_peek_ranges2d

In [9]:
### 求一行的中位数宽度
def compute_median_w_from_ranges(peek_ranges):
    widthes = []
    for peek_range in peek_ranges:
        w = peek_range[1] - peek_range[0] + 1
        widthes.append(w)
    widthes = np.asarray(widthes)
    median_w = np.median(widthes)
    return median_w

In [10]:
def combine_char(peek_ranges, vertical_peek_ranges2d, ratio = 0.8):##eg:似
    filtered_vertical_peek_ranges2d = []
    for i, peek_range in enumerate(peek_ranges):
        new_peek_range = []
        median_w = compute_median_w_from_ranges(vertical_peek_ranges2d[i])
        stan_w = median_w * ratio
        last_w = None#用来记录上一个区间的宽度
        length = len(vertical_peek_ranges2d[i])
        for j, vertical_range in enumerate(vertical_peek_ranges2d[i]):
            cur_w = vertical_range[1] - vertical_range[0]
            if cur_w < stan_w and cur_w > median_w*0.1:
                if last_w != None:
                    if cur_w > median_w*0.5:
                        new_node = (vertical_peek_ranges2d[i][j-1][0],vertical_peek_ranges2d[i][j][1])#连续两个非正常区间合并
                        new_peek_range.append(new_node)
                        last_w = None
                    else:
                        new_node_last = (vertical_peek_ranges2d[i][j-1][0],vertical_peek_ranges2d[i][j-1][1])#连续两个非正常区间合并
                        new_peek_range.append(new_node_last)
                        new_node_new = (vertical_peek_ranges2d[i][j][0],vertical_peek_ranges2d[i][j][1])#连续两个非正常区间合并
                        new_peek_range.append(new_node_new)
                else:
                    last_w = True
            else:
                if last_w != None:
                    new_node_last = (vertical_peek_ranges2d[i][j-1][0],vertical_peek_ranges2d[i][j-1][1])#连续两个非正常区间合并
                    new_peek_range.append(new_node_last)
                    new_node_new = (vertical_peek_ranges2d[i][j][0],vertical_peek_ranges2d[i][j][1])#连续两个非正常区间合并
                    new_peek_range.append(new_node_new)
                else:
                    
                    new_peek_range.append(vertical_range)
                last_w = None
            if j+1 == length and cur_w < stan_w and last_w==True:
                new_peek_range.append(vertical_range)
        filtered_vertical_peek_ranges2d.append(new_peek_range)
    return filtered_vertical_peek_ranges2d


In [11]:
def crop_regular(peek_ranges, vertical_peek_ranges2d, adaptive_image):
    filtered_vertical_peek_ranges2d = []
    for i, peek_range in enumerate(peek_ranges):
        new_peek_range = []
        for j, vertical_range in enumerate(vertical_peek_ranges2d[i]):
            start_x = peek_range[0]
            end_x = peek_range[1]
            line_img = adaptive_image[start_x:end_x, :]
            vertical_sum = np.sum(line_img, axis=0)
            vertical_peek_range = extract_peek_ranges_from_array_single(vertical_sum)
            new_peek_range.append(vertical_peek_range)
    filtered_vertical_peek_ranges2d.append(new_peek_range)

In [12]:
class CaffeCls(object):
    def __init__(self, 
                 model_def,
                 model_weights,
                 y_tag_json_path,
                 is_mode_cpu=True,
                 width=64,
                 height=64):
        self.net = caffe.Net(model_def,
            model_weights,
            caffe.TEST)
        if is_mode_cpu:
            caffe.set_mode_cpu()
        self.y_tag_json = json.load(open(y_tag_json_path, "r"))
        self.width = width
        self.height = height

    def _predict_cv2_imgs_sub(self, cv2_imgs, pos_start, pos_end):
        cv2_imgs_sub = cv2_imgs[pos_start: pos_end]
        #if cv2_imgs_sub[0][0][0] > 1.0:
            #raise ValueError("image should be normalized between 0 and 1.0")
        self.net.blobs['data'].reshape(cv2_imgs_sub.shape[0], 1,
                                       self.width, self.height)
        self.net.blobs['data'].data[...] = cv2_imgs_sub.reshape(
            (cv2_imgs_sub.shape[0], 1, self.width, self.height))

        output = self.net.forward()
        output_tag_to_max_proba = []
        
        num_sample = cv2_imgs_sub.shape[0]
        for i in range(num_sample):
            output_prob = output['prob'][i]
            output_prob_index = sorted(
                range(len(output_prob)),
                key=lambda x:output_prob[x],
                reverse=True)            
            output_tag_to_probas = []
            for index in output_prob_index:
                item = (self.y_tag_json[str(index)],
                        output_prob[index])
                output_tag_to_probas.append(item)
            # output_tag_to_probas = output_tag_to_probas[:2]
            output_tag_to_max_proba.append(output_tag_to_probas)
        return output_tag_to_max_proba

    def predict_cv2_imgs(self, cv2_imgs, step=50):
        output_tag_to_max_proba = []
        num_sample = cv2_imgs.shape[0]
        for i in range(0, num_sample, step):
            pos_end = min(num_sample, (i + step))
            output_tag_to_max_proba += \
                self._predict_cv2_imgs_sub(cv2_imgs, i, pos_end)
        return output_tag_to_max_proba

    def predict_cv2_img(self, cv2_img):
        shape = cv2_img.shape
        cv2_imgs = cv2_img.reshape((1, shape[0], shape[1]))
        return self.predict_cv2_imgs(cv2_imgs)[0]

In [13]:
class PreprocessCropZeros(object):#四个方向切一遍

    def __init__(self):
        pass

    def do(self, cv2_gray_img):
        height = cv2_gray_img.shape[0]
        width = cv2_gray_img.shape[1]

        v_sum = np.sum(cv2_gray_img, axis=0)
        h_sum = np.sum(cv2_gray_img, axis=1)
        left = 0
        right = width - 1
        top = 0
        low = height - 1

        for i in range(width):
            if v_sum[i] > 30:
                left = i
                break

        for i in range(width - 1, -1, -1):
            if v_sum[i] > 300:
                right = i
                break

        for i in range(height):
            if h_sum[i] > 30:
                top = i
                break

        for i in range(height - 1, -1, -1):
            if h_sum[i] > 30:
                low = i
                break
        if not (top < low and right > left):
            return cv2_gray_img

        return cv2_gray_img[top: low+1, left: right+1]

In [14]:
class PreprocessResizeKeepRatioFillBG(object):

    def __init__(self, width, height, fill_bg=False,
                 auto_avoid_fill_bg=True, margin=None):
        self.width = width
        self.height = height
        self.fill_bg = fill_bg
        self.auto_avoid_fill_bg = auto_avoid_fill_bg
        self.margin = margin

    @classmethod
    def is_need_fill_bg(cls, cv2_img, th=0.5, max_val=255):
        image_shape = cv2_img.shape
        height, width = image_shape
        if height * 3 < width:
            return True
        if width * 3 < height:
            return True
        return False

    @classmethod
    def put_img_into_center(cls, img_large, img_small, ):
        width_large = img_large.shape[1]
        height_large = img_large.shape[0]

        width_small = img_small.shape[1]
        height_small = img_small.shape[0]

        if width_large < width_small:
            #raise ValueError("width_large <= width_small")
            return img_large
        if height_large < height_small:
            #raise ValueError("height_large <= height_small")
            return img_large

        start_width = (width_large - width_small) / 2
        start_height = (height_large - height_small) / 2

        img_large[start_height:start_height + height_small,
                  start_width:start_width + width_small] = img_small
        return img_large

    def do(self, cv2_img):

        if self.margin is not None:
            width_minus_margin = max(2, self.width - self.margin)
            height_minus_margin = max(2, self.height - self.margin)
        else:
            width_minus_margin = self.width
            height_minus_margin = self.height

        cur_height, cur_width = cv2_img.shape[:2]
        if len(cv2_img.shape) > 2:
            pix_dim = cv2_img.shape[2]
        else:
            pix_dim = None

        preprocess_resize_keep_ratio = PreprocessResizeKeepRatio(
            width_minus_margin,
            height_minus_margin)
        resized_cv2_img = preprocess_resize_keep_ratio.do(cv2_img)

        if self.auto_avoid_fill_bg:
            need_fill_bg = self.is_need_fill_bg(cv2_img)
            if not need_fill_bg:
                self.fill_bg = False
            else:
                self.fill_bg = True

        ## should skip horizontal stroke
        if not self.fill_bg:
            ret_img = cv2.resize(resized_cv2_img, (width_minus_margin,
                                                   height_minus_margin))
        else:
            if pix_dim is not None:
                norm_img = np.zeros((height_minus_margin,
                                     width_minus_margin,
                                     pix_dim),
                                    np.uint8)
            else:
                norm_img = np.zeros((height_minus_margin,
                                     width_minus_margin),
                                    np.uint8)
            ret_img = self.put_img_into_center(norm_img, resized_cv2_img)

        if self.margin is not None:
            if pix_dim is not None:
                norm_img = np.zeros((self.height,
                                     self.width,
                                     pix_dim),
                                    np.uint8)
            else:
                norm_img = np.zeros((self.height,
                                     self.width),
                                    np.uint8)
            ret_img = self.put_img_into_center(norm_img, ret_img)
        return ret_img

In [15]:
def test_predict(test_image, caffe_cls_number, caffe_cls_region):
    #读取
    #test_image = "/opt/plateRec/example/example_3.jpg"
    raw_color_img = plt.imread(test_image)
    #plt.figure(figsize=(15,10))

    #放大
    resize_keep_ratio = PreprocessResizeKeepRatio(1024, 1024)
    color_img = resize_keep_ratio.do(raw_color_img)
    #plt.figure(figsize=(15,10))
    #plt.imshow(color_img)
    #灰度
    gray_image = cv2.cvtColor(color_img, cv2.COLOR_RGB2GRAY)

    _, adaptive_image = cv2.threshold(
            gray_image,
            120,
            255,\
            cv2.THRESH_BINARY)
    
    vertical_peek_ranges2d = []
    horizontal_sum = np.sum(adaptive_image, axis=1)
    peek_ranges = extract_peek_ranges_from_array_x(horizontal_sum)
    
     

    vertical_sum = np.sum(adaptive_image, axis=0)    
    
    vertical_peek_ranges2d = crop_direct(peek_ranges, adaptive_image, debug = False) 
    draw_rectangle(color_img, peek_ranges, vertical_peek_ranges2d)
  
    char_imgs_region = []
    char_imgs_number = []
    crop_zeros = PreprocessCropZeros()
    resize_keep_ratio = PreprocessResizeKeepRatioFillBG(
        norm_width, norm_height, fill_bg=False, margin=4)
    for i, peek_range in enumerate(peek_ranges):
        for j,vertical_range in enumerate(vertical_peek_ranges2d[i]):
            x = vertical_range[0]
            y = peek_range[0]
            w = vertical_range[1] - x
            h = peek_range[1] - y
            if j == 0: # 第一个字符
                char_img_region = adaptive_image[y:y+h+1, x:x+w+1]
                char_img_region = crop_zeros.do(char_img_region)
                char_img_region = resize_keep_ratio.do(char_img_region)
                
                char_imgs_region.append(char_img_region)
            else:               
                char_img_number = adaptive_image[y:y+h+1, x:x+w+1]
                char_img_number = crop_zeros.do(char_img_number)
                char_img_number = resize_keep_ratio.do(char_img_number)
                char_imgs_number.append(char_img_number)

    np_char_imgs_region = np.asarray(char_imgs_region)
    np_char_imgs_region = np_char_imgs_region / 255.0
    np_char_imgs_number = np.asarray(char_imgs_number)
    np_char_imgs_number = np_char_imgs_number / 255.0
    
   
    output_tag_to_max_proba_number = caffe_cls_number.predict_cv2_imgs(np_char_imgs_number)
    output_tag_to_max_proba_region = caffe_cls_region.predict_cv2_imgs(np_char_imgs_region)
    ocr_res_region = ""
    for item in output_tag_to_max_proba_region:
        ocr_res_region += item[0][0]  
    ocr_res_number = ""
    for item in output_tag_to_max_proba_number:
            ocr_res_number += item[0][0]


In [16]:
if __name__ == "__main__":

    norm_width = 64
    norm_height = 64
    test_path = '/workspace/plateRec/test/'
    
    base_dir_number = "/workspace/plateRec/number"
    model_def_number = os.path.join(base_dir_number, "lenet.prototxt")
    model_weights_number = os.path.join(base_dir_number, "lenet_iter_1000.caffemodel")
    y_tag_json_path_number = os.path.join(base_dir_number, "y_tag.json")
    caffe_cls_number = CaffeCls(model_def_number, model_weights_number, y_tag_json_path_number)
    
    base_dir_region = "/workspace/plateRec/region"
    model_def_region = os.path.join(base_dir_region, "lenet.prototxt")
    model_weights_region = os.path.join(base_dir_region, "lenet_iter_1000.caffemodel")
    y_tag_json_path_region = os.path.join(base_dir_region, "y_tag.json")
    caffe_cls_region = CaffeCls(model_def_region, model_weights_region, y_tag_json_path_region)
    
    for test_image in os.listdir(test_path):
        %time test_predict(test_image=test_path+test_image, caffe_cls_number=caffe_cls_number, caffe_cls_region=caffe_cls_region)

CPU times: user 80 ms, sys: 10 ms, total: 90 ms
Wall time: 94.7 ms
CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 75 ms
CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 64.3 ms
CPU times: user 70 ms, sys: 0 ns, total: 70 ms
Wall time: 61.9 ms
CPU times: user 80 ms, sys: 10 ms, total: 90 ms
Wall time: 94.9 ms
CPU times: user 50 ms, sys: 0 ns, total: 50 ms
Wall time: 54.5 ms
CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 55.8 ms
CPU times: user 90 ms, sys: 0 ns, total: 90 ms
Wall time: 87.8 ms
CPU times: user 50 ms, sys: 10 ms, total: 60 ms
Wall time: 60.7 ms
CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 57.4 ms
CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 98.1 ms
CPU times: user 50 ms, sys: 10 ms, total: 60 ms
Wall time: 64.6 ms
CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 63.4 ms
CPU times: user 90 ms, sys: 10 ms, total: 100 ms
Wall time: 98 ms
CPU times: user 80 ms, sys: 0 ns, total: 80 ms
Wall time: 80.6 ms
